# All in one

In [9]:
from skimage import io, morphology, color, util, filters
import numpy as np
import pandas as pd
import os
import shutil
from PIL import Image

import matplotlib.pyplot as plt
#sns.set(style="darkgrid", palette="Set2")

# height and width in pixels
tile_size = 300

import warnings
warnings.filterwarnings("ignore")

In [10]:
img_folder = "data/"

img_overview_fname = "image_overview.csv"
img_overview = pd.read_csv(img_folder + img_overview_fname)

# create dir
if os.path.exists("processed_data/"):
    shutil.rmtree("processed_data")
tile_dir = "processed_data/tiles/"
os.makedirs(tile_dir, exist_ok=True)

# tile csv
d = {"input_tile": list(), "target_tile": list()}

In [11]:
for k in range(len(img_overview)):
    # read fname and img
    input_img_fname = img_overview.at[k, "input_img_fname"]
    target_img_fname = img_overview.at[k, "segmentation_fname"]

    input_img  = io.imread(img_folder + input_img_fname)
    target_img  = io.imread(img_folder + target_img_fname)

    # get dimensions
    if len(input_img.shape) != 2:
        input_img = (color.rgb2gray(input_img)*255).astype(np.uint8)
    
    img_h, img_w = input_img.shape
    pad_size_h, pad_size_w = int((tile_size - (img_h%tile_size))), int((tile_size - (img_w%tile_size)))
    n_tiles_h, n_tiles_w = img_h//tile_size + 1, img_w//tile_size + 1
    pad_img_h, pad_img_w = n_tiles_h*tile_size, n_tiles_w*tile_size

    # pad for whole number of tiles, 3d for last img
    input_padded = np.pad(input_img, ((0, pad_size_h), (0, pad_size_w)), 'constant', constant_values=False)
    target_padded = np.pad(target_img, ((0, pad_size_h), (0, pad_size_w), (0,0)), 'constant', constant_values=False)

    # save tiles
    background_mean = np.mean(target_padded[0:tile_size, 0:tile_size])
    
    for i in range(0, pad_img_h, tile_size):
        for j in range(0, pad_img_w, tile_size):
            # if similar to background, dont include, most likely no mycelium
            if np.mean(target_padded[i:i+tile_size, j:j+tile_size]) < (background_mean + 1):
                continue
            
            fname_append = f"img-{k}_h-{i}_w-{j}"
            input_tile_fname = "brightfield_" + fname_append + ".jpg"
            target_tile_fname = "fluorescence_" + fname_append + ".jpg"

            io.imsave(tile_dir + input_tile_fname, input_padded[i:i+tile_size, j:j+tile_size], check_contrast=False)
            io.imsave(tile_dir + target_tile_fname, target_padded[i:i+tile_size, j:j+tile_size], check_contrast=False)
            
            d["input_tile"].append(input_tile_fname)
            d["target_tile"].append(target_tile_fname)

tile_overview = pd.DataFrame(d)
tile_overview.to_csv(tile_dir + "tile_overview.csv")